In [143]:
import numpy as np
import pandas as pd
import glob, os
from scipy.stats import norm
import matplotlib.pyplot as plt

--- **Data Aggregation and Cleaning were here** ---

Initially this script included lines meant for data aggregation and cleaning. As the process is going to be the same for all upcoming scripts, these lines are deleted. However, they can be found in the first script related to Safety and Cycle Stock calculation.

--- **Fill Rate Simulation script starts here** ---

Product chosen for this script is one of supplier's topsellers. It's is rarely included in promo activities.

In [162]:
dfPivot = dfPivot[dfPivot['Base_Unit_Code']=='1234567890']

Data sample for the chosen product is displayed below.

In [163]:
print(dfPivot.sample(5))

Line_ID Movement_Week Movement_Date       EAN_Code Base_Unit_Code  Stock  Transit  Shipped   OOS  Demand
1759             cw49    2020-12-01  4600000000001     1234567890  12434    64800    10313     0   10313
3204             cw43    2020-10-21  4600000000001     1234567890  63083    17712     6993     0    6993
626              cw53    2021-01-02  4600000000001     1234567890  27557     6048    16146     0   16146
1905             cw48    2020-11-27  4600000000001     1234567890  23315        0    16587  2835   19422
2470             cw46    2020-11-11  4600000000001     1234567890  66632        0     7938     0    7938


We choose calendar day as a period. 

In [164]:
dfOneProduct = dfPivot.groupby('Movement_Date',as_index=False)[['Stock','Shipped','OOS','Demand']].sum()

Aggregated quantities can be observed in the following table.

In [165]:
print(dfOneProduct[['Movement_Date','Demand']].tail(10))

Line_ID Movement_Date  Demand
148        2021-01-10    6237
149        2021-01-11    6228
150        2021-01-12    3636
151        2021-01-13    4203
152        2021-01-14   11430
153        2021-01-15    5931
154        2021-01-16    1944
155        2021-01-17   10404
156        2021-01-18   18999
157        2021-01-19   25766


*Z* is a demand threshold. Below we will calculate the probability for an occurence of demand quantity to be below *Z* value within a chosen period. I have set 15.000 as a demand threshold as it almost equals to the inverse CDF of a normal distribution.

In [166]:
z = 15000 # Chosen value is almost equal to the inverse CDF of a normal distribution.

*MU* is a mean value for product's demand. In our case it's calculated based on data since August 16th.

In [167]:
mu = np.round(dfOneProduct['Demand'].mean(),decimals=2) # mean value

*Sigma (std)* is standard deviation value for product's demand.

In [168]:
std = np.round((np.std(dfOneProduct['Demand'],ddof=1)),decimals=2) # standard deviation or std

In [169]:
print('\n','z:',z,'\n','mu:',mu,'\n','std:',std)


 z: 15000 
 mu: 7361.31 
 std: 4388.74


Coefficients below were found by Andrade and Sikorski. These numbers let you calculate inverse standard normal loss function.

In [170]:
coefficients = [ 4.41738119e-09, 1.79200966e-07, 3.01634229e-06,
2.63537452e-05, 1.12381749e-04, 5.71289020e-06,
-2.64198510e-03, -1.59986142e-02, -5.60399292e-02,
-1.48968884e-01, -3.68776346e-01, -1.22551895e+00,
-8.99375602e-01]

def inverse_standard_loss(target):
    x = np.log(target)
    z = np.polyval(coefficients, x)
    return z

In [198]:
time = 10000    # Time period chosen for simulation.

L = 2           # Lead time chosen for simulation.
L_std = 0       # No deviation for lead time is added.
R = 7           # Replenishment happens every 7 days.

x_std = np.sqrt((L+R)*std**2+L_std**2*mu**2)

d = np.maximum(np.random.normal(mu,std,time).round(0).astype(int),0)

In [172]:
beta = 0.98 # the desired fill rate
d_c = mu * R
target = d_c*(1-beta)/x_std
z = inverse_standard_loss(target) # The service level factor.
alpha = round(norm.cdf(z),3)      # The expected cycle service level.

In [174]:
Ss = np.round(x_std*z).astype(int) 
Ss = Ss - d_c*(1-beta) # Excess demand result in lost sales rather than backorders.
Cs = 1/2 * mu * R      # Cycle stock.
Is = mu * L            # Stock in transit.
S = Ss + 2*Cs + Is     # Overall stock.

In [175]:
hand = np.zeros(time,dtype=int)                   # numpy array for stock on hand, that will be updated in for loop
transit = np.zeros((time,L+7*L_std+1),dtype=int)  # numpy array for stock in transit, that will be updated in for loop

hand[0] = S - d[0]
transit[0,L] = d[0]

In [176]:
stockout_period = np.full(time,False,dtype=bool)
stockout_cycle = []
unit_shorts = np.zeros(time,dtype=int)

In [177]:
for t in range(1,time):
    if transit[t-1,0]>0:
        stockout_cycle.append(stockout_period[t-1])
    unit_shorts[t] = max(0,d[t] - max(0,hand[t-1] + transit[t-1,0]))
    hand[t] = hand[t-1] - d[t] + transit[t-1,0]
    stockout_period[t] = hand[t] < 0
    hand[t] = max(0,hand[t]) #Uncomment if excess demand result in lost sales rather than backorders
    transit[t,:-1] = transit[t-1,1:]        
    if t%R==0:
        actual_L = int(round(max(np.random.normal(L,L_std),0),0))
        net = hand[t] + transit[t].sum()    
        transit[t,actual_L] = S - net

In [178]:
print("Alpha:",round(alpha*100,1))
print("Beta:",beta*100,'\n')

fill_rate = 1-unit_shorts.sum()/sum(d)
print("Fill Rate:",round(fill_rate*100,1))

SL_alpha = 1-sum(stockout_cycle)/len(stockout_cycle)
print("Cycle Service Level:",round(SL_alpha*100,1))

SL_period = 1-sum(stockout_period)/time
print("Period Service Level:",round(SL_period*100,1))

Alpha: 84.9
Beta: 98.0 

Fill Rate: 98.2
Cycle Service Level: 86.0
Period Service Level: 97.3


According to the cycle service level, we have rather a high possibility (~ 14%) of facing Out of Stock by the end of the cycle (review period). However, the expected amount of goods short is rather low (~ 2%).

**Cycle Service Level** means the probability of facing OOS *during a review cycle*. In our case it's 7 days. Even if we face OOS only once during 7 days (for example, on the last day of cycle), we will have 0% service level for this cycle. In this case even 1 day of OOS can effect the overall service level in a greater degree. However, *it doesn't matter whether we had 1 or 2-3 cases of OOS within the cycle*. Its cycle service level is already 0%.

Idea for that logic is that we have less periods for estimation: [number of days] / [length of cycle].

We're likely to be short of goods during a cycle for the following number of times:

In [195]:
print(sum(stockout_cycle))

200


**Period Service Level** means the probability of facing OOS *over the whole simulated period*. In our case it's 10000 days.

As opposed to cycle service level, one case of OOS doesn't affect the service level so much. *OOS can drop the service level for one day only - when it actually happens*. However, in case of cycle service level, one OOS drops the whole cycle's service level.

We are going to have the following number of OOS within the mentioned period. All these cases are distributed over cycles mentioned above. If we have more OOS cased during period than during cycle, it means that in some cycles we had more than 1 OOS. For example, during two last days of cycle before new replenishment nothing could be sold because of OOS. 

In [196]:
print(sum(stockout_period))

267
